In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
import scipy
from scipy.optimize import curve_fit
from scipy import interpolate



from matplotlib import pyplot as plt
from scipy import constants
from numpy.polynomial import Polynomial
import pandas as pd
import matplotlib.colors as colors 
import seaborn as sns
import pandas as pd

In [ ]:
array_data = "/Users/proden/Code/SCQED-PCQED/array_data"
cavity_E_array_0_00 = np.load(array_data + "/H2/fci_cavity_arrays_H2_6311G0.npy")
cavity_E_array_0_001 = np.load(array_data + "/H2/fci_cavity_arrays_H2_6311G0_001.npy")
cavity_E_array_0_005 = np.load(array_data + "/H2/fci_cavity_arrays_H2_6311G0_005.npy")
cavity_E_array_0_01 = np.load(array_data + "/H2/fci_cavity_arrays_H2_6311G0_01.npy")
cavity_E_array_0_02 = np.load(array_data + "/H2/fci_cavity_arrays_H2_6311G0_02.npy")
cavity_E_array_0_03 = np.load(array_data + "/H2/fci_cavity_arrays_H2_6311G0_03.npy")
cavity_E_array_0_04 = np.load(array_data + "/H2/fci_cavity_arrays_H2_6311G0_04.npy")
cavity_E_array_0_05 = np.load(array_data + "/H2/fci_cavity_arrays_H2_6311G0_05.npy")
r_data = np.load(array_data + "/H2/fci_r_array_H2.npy")
N_R = r_data.shape[0]

## Calculation of k
Fit ground state PES of H2 to a quintic polynomial

In [ ]:
def find_crossover(array1, array2):
    idx = (np.abs(array1 - array2)).argmin()

    return idx

def compute_full_array(array1, array2, idx1, idx2):
    index = find_crossover(array1[idx1,:], array2[idx2,:])
    array_1 = array1[idx1,:index]
    array_2 = array2[idx2,index:]
    array_full = np.concatenate((array_1, array_2), axis=0)    

    return array_full

S0_array = cavity_E_array_0_00[0,:]
S1_array = compute_full_array(cavity_E_array_0_00, cavity_E_array_0_00, 3, 2)

LP_0_00 = compute_full_array(cavity_E_array_0_00, cavity_E_array_0_00, 1, 2)
LP_0_001 = compute_full_array(cavity_E_array_0_001, cavity_E_array_0_001, 1, 2)
LP_0_005 = compute_full_array(cavity_E_array_0_005, cavity_E_array_0_005, 1, 2)
LP_0_01 = compute_full_array(cavity_E_array_0_01, cavity_E_array_0_01, 1, 2)
LP_0_02 = compute_full_array(cavity_E_array_0_02, cavity_E_array_0_02, 1, 2)
LP_0_03 = compute_full_array(cavity_E_array_0_03, cavity_E_array_0_03, 1, 2)
LP_0_04 = compute_full_array(cavity_E_array_0_04, cavity_E_array_0_04, 1, 2)
LP_0_05 = compute_full_array(cavity_E_array_0_05, cavity_E_array_0_05, 1, 2)

UP_0_00 = compute_full_array(cavity_E_array_0_00, cavity_E_array_0_00, 3, 4)
UP_0_001 = compute_full_array(cavity_E_array_0_001, cavity_E_array_0_001, 3, 4)
UP_0_005 = compute_full_array(cavity_E_array_0_005, cavity_E_array_0_005, 3, 4)
UP_0_01 = compute_full_array(cavity_E_array_0_01, cavity_E_array_0_01, 3, 4)
UP_0_02 = compute_full_array(cavity_E_array_0_02, cavity_E_array_0_02, 3, 4)
UP_0_03 = compute_full_array(cavity_E_array_0_03, cavity_E_array_0_03, 3, 4)
UP_0_04 = compute_full_array(cavity_E_array_0_04, cavity_E_array_0_04, 3, 4)
UP_0_05 = compute_full_array(cavity_E_array_0_05, cavity_E_array_0_05, 3, 4)

LP_arrays = np.array([LP_0_00, 
            LP_0_001,
            LP_0_005,
            LP_0_01,
            LP_0_02,
            LP_0_03,
            LP_0_04,
            LP_0_05]
             )

UP_arrays = np.array([UP_0_00, 
            UP_0_001,
            UP_0_005,
            UP_0_01,
            UP_0_02,
            UP_0_03,
            UP_0_04,
            UP_0_05]
             )

In [ ]:

min_LP_array = np.zeros((1,8))
min_UP_array = np.zeros((1,8))

for i in range(8):
    min_LP_array[0,i] = np.argmin(LP_arrays[i])
    min_UP_array[0,i] = np.argmin(UP_arrays[i])

min_S0_loc = np.argmin(S0_array)
min_S1_loc = np.argmin(S1_array)

r_eq_ang = r_data[min_S0_loc]
print(f'Min on S0 is {r_data[min_S0_loc]}')
print(f'Min on S1 is {r_data[min_S1_loc]}')
#print(f'Min on LP is {r_data[min_LP_loc]}')
#print(f'Min on UP is {r_data[min_UP_loc]}')

# Fitting S0 PES to a quintic polynomial
au_to_SI = (4.35974 * 10 ** (-18)) * 10 ** 20
poly = np.poly1d(np.polyfit(r_data, S0_array, 8))
print(poly)

#Taking first and second derivative of S0 PES and evaluating at r_eq
first_derivative = poly.deriv()
second_derivative = first_derivative.deriv()
k_test_au = second_derivative(r_eq_ang)
k_test_SI = k_test_au * au_to_SI
print(k_test_SI)

#plotting S0 PES and quintic fit
plt.plot(r_data, poly(r_data), 'm-', label='fit')
plt.plot(r_data, S0_array, 'bo', label='cavity free |g>')
plt.show

## Calculation of $ \text{x}_0 $

$$ \frac{\hbar}{2}\sqrt{\frac{k}{\mu}} = \frac{k}{2}(x_0 - x_{eq})^2 + V_0 $$

Expanded, solved for $ x_o $, and found zeros using quadratic formula

First plot the ground-state potential energy surfaces for $ \text{H2} $ inside and outisde the cavity.  The effect of the cavity will raise the energy slightly.

In [ ]:
lambda_list = [0, 0.001, 0.005, 0.01, 0.02, 0.03, 0.04, 0.05]
fci_S0_omega = np.zeros((1, N_R))
for i in range(N_R):
    fci_S0_omega[0,i] = S0_array[i] + 0.3944656448298627
fci_S0_plusw = np.ndarray.flatten(fci_S0_omega)

color_list_1 = ['grey','violet','indigo','blue','green','yellow','orange','red']

plt.plot(r_data, fci_S0_plusw, 'gold', label='cavity free |g + $\omega$>')
plt.plot(r_data, S1_array, 'turquoise', label='cavity free |e>')
for i in range(8):
    plt.plot(r_data, LP_arrays[i,:], f'{color_list_1[i]}', label=f'$\lambda$ = {lambda_list[i]}')
for k in range(8):
        plt.plot(r_data, UP_arrays[k,:], f'{color_list_1[k]}',)
plt.xlabel('r (Angstrom)')
plt.ylabel('E (Hartree)')


#plt.xlim(1.5, 2)
#plt.ylim(-7.76, -7.72)
plt.legend(loc = 'upper right', ncols = 3, fontsize = 'small')
plt.savefig('ppsurfaces.png',dpi=500)
#plt.show()

In [ ]:
plt.plot(r_data, S0_array + 0.47, color = 'grey', linestyle = 'dashed')
plt.plot(r_data, S1_array, 'grey', linestyle = 'dashed')
plt.show()


plt.plot(r_data, S0_array + 0.45, color = 'grey', linestyle = 'dashed')
plt.plot(r_data, S1_array, 'grey', linestyle = 'dashed')
plt.show()

plt.plot(r_data, S0_array + 0.39, color = 'grey', linestyle = 'dashed')
plt.plot(r_data, S1_array, 'grey', linestyle = 'dashed')
plt.show()

In [ ]:
#calculate some constants for LiH

amu_to_au = 1822.89


mA_kg = 1.00784 * (10 ** (-3) / (6.022 * 10 ** 23) )
mB_kg = 6.9410 * (10 ** (-3) / (6.022 * 10 ** 23) )
mA_au = 1.00784 * amu_to_au
mB_au = 6.9410 * amu_to_au
mu_au = (mA_au * mB_au )/ (mA_au + mB_au)
mu_kg = (mA_kg * mB_kg) / (mA_kg + mB_kg)  
print("mu_au: ", mu_au)

In [ ]:


hbar = 1

# number of grid points 
N = 2001


def get_fd_wfn(x, V_y: np.array, use_5_point_stencil = False):

    hbar = 1


    # define grid spacing h
    h = x[1]-x[0]

    # create arrays for T, V, and H - we truncate the smallest and largest grid points where 
    # the centered finite difference derivatives cannot be defined
    T = np.zeros((N-2, N-2))
    V = np.zeros((N-2, N-2))
    H = np.zeros((N-2, N-2))

    # this uses the 3 point stencil; we can adapt to use a 5 point and it might improve accuracy


    if not use_5_point_stencil:
        for i in range(N-2):
            for j in range(N-2):
                if i==j:
                    T[i,j]= -2
                elif np.abs(i-j)==1:
                    T[i,j]=1
                else:
                    T[i,j]=0

        T = -T *( hbar ** 2 / (2 * mu_au* h**2))
        #T =  (- (hbar ** 2) / (2* mu_kg)) *  (1 / ( h**2)) * joule_to_hartree  * T


    elif use_5_point_stencil:
        for i in range(N-2):
            for j in range(N-2):
                if i==j:
                    T[i,j]= -30
                elif np.abs(i-j)==1:
                    T[i,j]=16
                elif np.abs(i-j)==2:
                    T[i,j]=-1

        T = -T *  ((hbar ** 2) / (2* mu_au))*  (1 / ( 12 * h**2)) 


    for i in range(N-2):
        for j in range(N-2):
            if i==j:
                V[i,j]= V_y[i+1]
            else:
                V[i,j]=0
                
    H = T + V

    #print((-T * hbar ** 2 / (2 * mu_kg* h**2)) * (2.294 * 10 ** 17))
    #print(V)

    vals, vecs = np.linalg.eigh(H)

    if np.average(vecs[:, 0]) < 0:
        vecs = vecs * -1

    return vals, vecs

In [ ]:
#FCF calculator
def get_fcf_matrix(potential_1, potential_2, r_data, matrix_size = 6, potential_1_is_groundstate = False):

    #geneerate 2 sets of wavefunctions for the potentials

    r_data_au = r_data / psi4.constants.bohr2angstroms

    min_potential_1_loc = np.argmin(potential_1[:])
    r_eq_au =r_data_au[potential_1.argmin()]

    #print("r_eq_au : " , r_eq_au)


    # Fitting S0 PES to a quintic polynomial

    poly = np.poly1d(np.polyfit(r_data_au, potential_1, 9))

    poly_array = np.asarray(poly)


    #Taking first and second derivative of S0 PES and evaluating at r_eq
    first_derivative = poly.deriv()
    second_derivative = first_derivative.deriv()
    k_au = second_derivative(r_eq_au)
    #print("k_au: ", k_au)


    angstrom_to_bohr = 1.88973
    x_min = r_data_au[0]
    x_max = r_data_au[-1]

    hbar = 1

    # number of grid points 
    N = 2001
    # define grid
    x = np.linspace(x_min, x_max, N)

    V_y = np.polyval(np.asarray(poly), (x))


    vals1, vecs1 = get_fd_wfn(x, V_y, use_5_point_stencil=True)


    # fig, ax1 = plt.subplots()

    # color = 'tab:red'
    # ax1.set_xlabel('bond length (m)')
    # ax1.set_ylabel('wfn', color=color)
    # ax1.plot(x[1:N-1], vecs1[:,0], 'r', label = "$\psi_0$")
    # ax1.plot(x[1:N-1], vecs1[:,1], 'b',label = "$\psi_1$" )
    # ax1.plot(x[1:N-1], vecs1[:,2], 'g',label = "$\psi_2$")
    # ax1.tick_params(axis='y', labelcolor=color)

    # ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    # color = 'tab:blue'
    # ax2.set_ylabel('energy (hartree)', color=color)  # we already handled the x-label with ax1
    # ax2.plot(r_data_au, potential_1, 'bo', label='PES_1')
    # ax2.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    # ax2.tick_params(axis='y', labelcolor=color)

    # fig.tight_layout()  # otherwise the right y-label is slightly clipped
    # fig.legend()
    # plt.show()


    min_potential_2_loc = np.argmin(potential_2[:])
    r_eq_au =r_data_au[min_potential_2_loc]

    #print("r_eq_au : " , r_eq_au)


    # Fitting S0 PES to a quintic polynomial

    #can use this line to only fit to bottom of well for cubic and harmonic
    #poly = np.poly1d(np.polyfit(r_data_meters[50:100], fci_S0[50:100], 4))

    poly = np.poly1d(np.polyfit(r_data_au, potential_2, 9))

    poly_array = np.asarray(poly)


    #Taking first and second derivative of S0 PES and evaluating at r_eq
    first_derivative = poly.deriv()
    second_derivative = first_derivative.deriv()
    k_au = second_derivative(r_eq_au)
    #print("k_au: ", k_au)


    angstrom_to_bohr = 1.88973
    x_min = r_data_au[0]
    x_max = r_data_au[-1]

    hbar = 1

    # number of grid points 
    N = 2001
    # define grid
    x = np.linspace(x_min, x_max, N)

    V_y = np.polyval(np.asarray(poly), (x))


    vals2, vecs2 = get_fd_wfn(x, V_y, use_5_point_stencil=True)

    # fig, ax1 = plt.subplots()

    # color = 'tab:red'
    # ax1.set_xlabel('bond length')
    # ax1.set_ylabel('wfn', color=color)
    # ax1.plot(x[1:N-1], vecs2[:,0], 'r',label = "$\psi_0$")
    # ax1.plot(x[1:N-1], vecs2[:,1], 'b',label = "$\psi_1$")
    # ax1.plot(x[1:N-1], vecs2[:,2], 'g', label = "$\psi_2$")

    # ax1.tick_params(axis='y', labelcolor=color)

    # ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    # color = 'tab:blue'
    # ax2.set_ylabel('energy (hartree)', color=color)  # we already handled the x-label with ax1
    # ax2.plot(r_data_au, potential_2, 'bo', label='PES_2')
    # ax2.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    # ax2.tick_params(axis='y', labelcolor=color)

    # fig.tight_layout()  # otherwise the right y-label is slightly clipped
    # fig.legend()
    # plt.show()


    FCF_matrix = np.zeros((matrix_size,matrix_size))

    for i in range(FCF_matrix.shape[0]):
        for j in range(FCF_matrix.shape[0]):

            FCF_matrix[i][j] = np.trapz(vecs1[:,i] * vecs2[:,j]) 
            FCF = np.absolute(FCF_matrix) ** 2 

    return FCF


In [ ]:
#FCF calculator
def get_fcf_matrix_approx(potential_1, potential_2, r_data, approx  = "harmonic", matrix_size = 6, potential_1_is_groundstate = False):

    #geneerate 2 sets of wavefunctions for the potentials

    r_data_au = r_data / psi4.constants.bohr2angstroms

    min_potential_1_loc = np.argmin(potential_1[:])
    r_eq_au =r_data_au[potential_1.argmin()]


    #print("r_eq_au : " , r_eq_au)


    # Fitting S0 PES to a quintic polynomial

    poly = np.poly1d(np.polyfit(r_data_au, potential_1, 9))

    poly_array = np.asarray(poly)


    #Taking first and second derivative of S0 PES and evaluating at r_eq
    first_derivative = poly.deriv()
    second_derivative = first_derivative.deriv()
    k_au = second_derivative(r_eq_au)
    De = np.abs(potential_1[-1] - potential_1[min_potential_1_loc])
    a = np.sqrt(k_au / (2*De))
    #print("k_au: ", k_au)


    angstrom_to_bohr = 1.88973
    x_min = r_data_au[0]
    x_max = r_data_au[-1]

    hbar = 1

    # number of grid points 
    N = 2001
    # define grid
    x = np.linspace(x_min, x_max, N)


    if approx == "harmonic":
        V_y = 0.5 * k_au  * (x-r_eq_au)**2 
    elif approx == "morse":
        V_y = De * ( 1- np.exp(- a * (x-r_eq_au))) ** 2

    vals1, vecs1 = get_fd_wfn(x, V_y, use_5_point_stencil=True)


    # fig, ax1 = plt.subplots()

    # color = 'tab:red'
    # ax1.set_xlabel('bond length (m)')
    # ax1.set_ylabel('wfn', color=color)
    # ax1.plot(x[1:N-1], vecs1[:,0], 'r', label = "$\psi_0$")
    # ax1.plot(x[1:N-1], vecs1[:,1], 'b',label = "$\psi_1$" )
    # ax1.plot(x[1:N-1], vecs1[:,2], 'g',label = "$\psi_2$")
    # ax1.tick_params(axis='y', labelcolor=color)

    # ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    # color = 'tab:blue'
    # ax2.set_ylabel('energy (hartree)', color=color)  # we already handled the x-label with ax1
    # ax2.plot(r_data_au, potential_1, 'bo', label='PES_1')
    # ax2.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    # ax2.tick_params(axis='y', labelcolor=color)

    # fig.tight_layout()  # otherwise the right y-label is slightly clipped
    # fig.legend()
    # plt.show()


    min_potential_2_loc = np.argmin(potential_2[:])
    r_eq_au =r_data_au[min_potential_2_loc]

    #print("r_eq_au : " , r_eq_au)


    # Fitting S0 PES to a quintic polynomial

    poly = np.poly1d(np.polyfit(r_data_au, potential_2, 9))

    poly_array = np.asarray(poly)


    #Taking first and second derivative of S0 PES and evaluating at r_eq
    first_derivative = poly.deriv()
    second_derivative = first_derivative.deriv()
    k_au = second_derivative(r_eq_au)
    De = np.abs(potential_2[-1] - potential_2[np.argmin(potential_2)])
    a = np.sqrt(k_au / (2*De))
    #print("k_au: ", k_au)


    angstrom_to_bohr = 1.88973
    x_min = r_data_au[0]
    x_max = r_data_au[-1]

    hbar = 1

    # number of grid points 
    N = 2001
    # define grid
    x = np.linspace(x_min, x_max, N)

    if approx == "harmonic":
        V_y = 0.5 * k_au  * (x-r_eq_au)**2 
    elif approx == "morse":
        V_y = De * ( 1- np.exp(- a * (x- r_eq_au))) ** 2



    vals2, vecs2 = get_fd_wfn(x, V_y, use_5_point_stencil=True)


    # fig, ax1 = plt.subplots()

    # color = 'tab:red'
    # ax1.set_xlabel('bond length')
    # ax1.set_ylabel('wfn', color=color)
    # ax1.plot(x[1:N-1], vecs2[:,0], 'r',label = "$\psi_0$")
    # ax1.plot(x[1:N-1], vecs2[:,1], 'b',label = "$\psi_1$")
    # ax1.plot(x[1:N-1], vecs2[:,2], 'g', label = "$\psi_2$")

    # ax1.tick_params(axis='y', labelcolor=color)

    # ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    # color = 'tab:blue'
    # ax2.set_ylabel('energy (hartree)', color=color)  # we already handled the x-label with ax1
    # ax2.plot(r_data_au, potential_2, 'bo', label='PES_2')
    # ax2.plot(r_data_au, poly(r_data_au), 'm-', label='fit')
    # ax2.tick_params(axis='y', labelcolor=color)

    # fig.tight_layout()  # otherwise the right y-label is slightly clipped
    # fig.legend()
    # plt.show()


    FCF_matrix = np.zeros((matrix_size,matrix_size))

    for i in range(FCF_matrix.shape[0]):
        for j in range(FCF_matrix.shape[0]):

            FCF_matrix[i][j] = np.trapz(vecs1[:,i] * vecs2[:,j]) 
            FCF = np.absolute(FCF_matrix) ** 2 

    return FCF


In [ ]:

print("lambda = 0.05")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[7,:],  r_data, approx = "harmonic",matrix_size= 5)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)


print("lambda = 0.04")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[6,:],  r_data, approx = "harmonic",matrix_size= 5)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



print("lambda = 0.03")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[5,:],  r_data, approx = "harmonic",matrix_size= 5)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)

print("lambda = 0.02")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[4,:],  r_data, approx = "harmonic",matrix_size= 5)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



print("lambda = 0.01")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[3,:],  r_data, approx = "harmonic",matrix_size= 5)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



print("lambda = 0.005")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[2,:],  r_data, approx = "harmonic",matrix_size= 5)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



print("lambda = 0.001")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[1,:],  r_data, approx = "harmonic",matrix_size= 5)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



In [ ]:

print("lambda = 0.05")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[7,:],  r_data, approx = "harmonic",matrix_size= 1)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)


print("lambda = 0.04")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[6,:],  r_data, approx = "harmonic",matrix_size= 1)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



print("lambda = 0.03")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[5,:],  r_data, approx = "harmonic",matrix_size= 1)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)

print("lambda = 0.02")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[4,:],  r_data, approx = "harmonic",matrix_size= 1)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



print("lambda = 0.01")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[3,:],  r_data, approx = "harmonic",matrix_size= 1)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



print("lambda = 0.005")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[2,:],  r_data, approx = "harmonic",matrix_size= 1)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)



print("lambda = 0.001")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[1,:],  r_data, approx = "harmonic",matrix_size= 1)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)




print("lambda = 0.00")
fcf_harmonic = get_fcf_matrix_approx(fci_S0_plusw,UP_arrays[0,:],  r_data, approx = "harmonic",matrix_size= 1)

"harmonic"
min_val, max_val = 0, 1
cm = sns.light_palette("blue", as_cmap=True)
x=pd.DataFrame(fcf_harmonic)
x=x.style.background_gradient(cmap=cm)
display(x)
